In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 2.7 MB/s 
     |████████████████████████████████| 895 kB 60.8 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 46.8 MB/s 
     |████████████████████████████████| 636 kB 73.8 MB/s 
     |████████████████████████████████| 109 kB 74.4 MB/s 
     |████████████████████████████████| 546 kB 73.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from torch import nn
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math
import time
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
torch.cuda.is_available()

True

In [ ]:
url = 'https://raw.githubusercontent.com/AdithyaOasis/Capstone/main/Data/DataSet_4.csv'
# url = 'https://raw.githubusercontent.com/AdithyaOasis/Capstone/main/Data/dataset_new.csv'

In [ ]:
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

In [ ]:
class BertEmbedding:
    def __init__(self):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
        # Put the model in "evaluation" mode, meaning feed-forward operation.
        self.model.eval()

    def getEmbeddings(self,text):
      tokens_tensor,segments_tensor = self.TextPreprocessing(text)
      embeddings = self.generateEmbedding(tokens_tensor,segments_tensor)
      return embeddings

    def TextPreprocessing(self,text):
        marked_text = "[CLS] " + text + " [SEP]"
        # Tokenize our sentence with the BERT tokenizer.
        tokenized_text = self.tokenizer.tokenize(marked_text)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        return tokens_tensor,segments_tensors
    
    def generateEmbedding(self,tokens_tensor,segments_tensor):
        with torch.no_grad():
            outputs = self.model(tokens_tensor, segments_tensor)
            hidden_states = outputs[2]
        tokens = torch.stack(hidden_states,dim=0)
        tokens = tokens.permute(1,2,0,3)
        final_val = tokens[:,1:-1,-2,:]
        return final_val

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
class MultiHeadedAttention(nn.Module):
    "Multi-headed Attention module"
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        # x, self.attn = attention(query, key, value, mask=mask)
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [ ]:
class FeedForwardLayer(nn.Module):
    "Feedforward with 1 hidden layer"
    def __init__(self,inp_dim,hid_dim,dropout = 0.1):
        super(FeedForwardLayer, self).__init__()
        self.inp_dim = inp_dim
        self.hid_dim = hid_dim
        self.hidden = nn.Linear(inp_dim,hid_dim)
        self.output = nn.Linear(hid_dim,inp_dim)
        self.relu = F.relu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output(x)
        #Can check with sigmoid addition
        return x

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
        # return (x-mean)/(std + self.eps)

In [ ]:
class SublayerConnection(nn.Module):
    "Apply residual connection to any sublayer with the same size."
    def __init__(self, size, dropout=0.1):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        
        return x + self.dropout(sublayer(self.norm(x)))
        # check = sublayer(self.norm(x));
        # print("sublayer:",check,"X:",x);
        # return x + check


In [ ]:
class EncoderBlock(nn.Module):
    "An Encoder block that find the connection between Answer and Question"
    def __init__(self,attentionBlock,feedForwardBlock,size,dropout = 0.1):
        super(EncoderBlock,self).__init__()
        self.attentionBlock = attentionBlock
        self.feedForwardBlock = feedForwardBlock
        self.sublayer = clones(SublayerConnection(size,dropout),2)
        self.size = size

    def forward(self,Query,Value):
        x = self.sublayer[0](Query, lambda x: self.attentionBlock(query=x, value=Value, key=Value))
        return self.sublayer[1](x, self.feedForwardBlock)


In [ ]:
class EncoderModule(nn.Module):
    "Stacks of Encoder blocks"
    def __init__(self, EncoderLayer,N):
        super(EncoderModule,self).__init__()
        self.layers = clones(EncoderLayer,N)
        self.norm = LayerNorm(EncoderLayer.size)  
    
    def forward(self,Query,Value):
        for layers in self.layers:
            Query = layers(Query = Query,Value = Value)
        return Query


In [ ]:
class RepresentationModule(nn.Module):
    "Bottom layer that gives 2 representations: Reference answer Rep and Student answer Rep"
    def __init__(self,EncoderModule, embeddingLayer):
        
        super(RepresentationModule,self).__init__()
        self.EncoderModules = clones(EncoderModule,2)
        self.embeddingLayer = embeddingLayer
        #Get 2 clones of EncoderModule
    def forward(self, Question, ReferenceAnswer, StudentAnswer):
        Q = self.embeddingLayer.getEmbeddings(Question)
        StuAns = self.embeddingLayer.getEmbeddings(StudentAnswer)
        RefAns = self.embeddingLayer.getEmbeddings(ReferenceAnswer)
        studentAnsRep = self.EncoderModules[0](Q,StuAns)
        RefAnsRep = self.EncoderModules[1](Q,RefAns)
        return (studentAnsRep,RefAnsRep)

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
        # return x


In [ ]:
class FinalFeedForwardLayer(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size,5000)
        self.fc2 = nn.Linear(5000,1)
    def forward(self, xb):
        xb = F.relu(self.fc1(xb))
        # xb = self.dropout(xb)
        return self.fc2(xb)
        # print(torch.count_nonzero(xb))
        # xb = torch.sigmoid(self.fc4(xb))      # batch wise forwarding
        # xb = F.relu(self.fc3(xb))
        # return xb

In [ ]:
class FinalClassificationLayer(nn.Module):
  #For 3 mark answers
    def __init__(self,input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size,5000)
        self.fc2 = nn.Linear(5000,4)
        # self.fc3 = nn.Linear(3000,1000)
        # self.fc4 = nn.Linear(1000,3)
        self.dropout = nn.Dropout(0.1)
    def forward(self, xb):
        xb = F.relu(self.fc1(xb))
        xb = self.dropout(xb)
        return self.fc2(xb)
        # xb = F.relu(self.fc1(xb))
        # xb = F.relu(self.fc2(xb))
        # xb = F.relu(self.fc3(xb))
        # xb = self.fc4(xb)      # batch wise forwarding
        # return xb

In [ ]:
class PrepLayer(nn.Module):
    def __init__(self,d_model,max_size,positionalLayer):
        super(PrepLayer,self).__init__()
        self.borderLayer = torch.zeros(1,1,768)
        self.d_model = d_model
        self.max_size = max_size
        self.positionalEncoding = positionalLayer
    def forward(self,StudAns,RefAns):
        RefAns = torch.add(RefAns,1)
        FinalRep = torch.cat((StudAns,self.borderLayer),dim=1)
        FinalRep = torch.cat((FinalRep,RefAns),dim = 1)
        FinalRep = self.positionalEncoding(FinalRep)
        FinalRep = FinalRep.flatten(start_dim=1, end_dim=2)
        # print("FinalLayerSize:",FinalRep.size())
        self.padValue = torch.zeros(1,self.d_model*(self.max_size*2+1) - FinalRep.shape[1])
        FinalRep = torch.cat((FinalRep,self.padValue),dim = 1)
        return FinalRep

In [ ]:
class GetMarks(nn.Module):
    #Grading Philosophy
    def __init__(self):
        super(GetMarks,self).__init__()
    def forward(self,grade,full_marks):
        # return grade*full_marks
        return grade

In [ ]:
class GradingModule(nn.Module):
    "Uses the Representations to compare and grade them"
    def __init__(self,d_model,max_size,prepLayer,feedForward):
        super(GradingModule,self).__init__()
        self.max_size = max_size
        self.d_model = d_model
        self.prepLayer = prepLayer
        self.feedForward = feedForward
        self.GetMarks = GetMarks()
        
    def forward(self,stu,ref,full_marks):
        FinalRep = self.prepLayer(stu,ref)
        grade = self.feedForward(FinalRep)
        final_marks = self.GetMarks(grade,full_marks)
        return final_marks
        

In [ ]:
class Upgrader(nn.Module):
    def __init__(self,Representation_Module, Grading_Module):
        super(Upgrader,self).__init__()
        self.RepModule = Representation_Module
        self.GradModule = Grading_Module
        self.stu_rep = 0;
        self.ref_rep = 0;
    def forward(self, StudentAns,Question,RefAnswer,full_marks):
        self.stu_rep,self.ref_rep = self.RepModule( Question,StudentAns,RefAnswer)
        grad = self.GradModule(self.stu_rep,self.ref_rep,full_marks)
        return grad

    def printRepresentations(self):
      print("Stu_rep:",self.stu_rep);
      print("Ref_rep:",self.ref_rep);

In [ ]:
def makeModel(emb_dim = 768,heads = 6,hid_lay_dim = 768, EncoderBlocksNo = 1,max_size = 17):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h=heads,d_model=emb_dim)
    ff = FeedForwardLayer(emb_dim,hid_lay_dim)
    embedding_layer = BertEmbedding()
    positionalLayer = PositionalEncoding(d_model=emb_dim)
    # FinalFF = FinalFeedForwardLayer(emb_dim*(max_size*2 + 1))
    FinalFF = FinalClassificationLayer(emb_dim*(max_size*2 + 1)) 
    model = Upgrader(
                RepresentationModule(
                    EncoderModule( EncoderBlock( c(attn), c(ff), emb_dim), EncoderBlocksNo),
                    embedding_layer)
                ,GradingModule(
                    d_model=emb_dim,
                    max_size=max_size,
                    prepLayer=PrepLayer(d_model = emb_dim,
                              max_size = max_size,
                              positionalLayer = positionalLayer),
                    feedForward = FinalFF
                ))
    return model

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = makeModel(heads = 6, EncoderBlocksNo = 3,max_size = 33).to(device)
model = makeModel(heads = 6, EncoderBlocksNo = 3,max_size = 33)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# model = makeModel(heads = 6, EncoderBlocksNo = 3,max_size = 33)
#Can change heads(Range(1,6)), EncoderBlocksNo (Range(1,6))

In [ ]:
df = pd.read_csv(url)

In [ ]:
# def training(model,X,Y,learningRate,epochs, breakOutLimit, breakOut,batchSize):
#   optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
#   criterion = nn.MSELoss()
#   loss_per_batch = []
#   loss_in_batch = []
#   for _ in range(epochs):
#         batchCount = 0
#         loss_in_batch.clear()
#         optimizer.zero_grad()
#         for i in range(len(X)):
#           # model.train()
#           Y_hat = model(X.iloc[i][0],X.iloc[i][1],X.iloc[i][2],X.iloc[i][3])
#           #X[0] = StudentAns, X[1] = Question, X[2] = Ref Ans, X[3] = Fullmarks
#           print("Yhat:",Y_hat," Y:",Y.iloc[i])
#           loss = criterion(Y_hat.float(),torch.tensor([[Y.iloc[i]]]).float())
#           # loss = loss.float()
#           print("Loss:",loss,"Type:",type(loss))
#           loss_in_batch.append(loss)
          
#           loss.backward()
#           batchCount += 1
#           if(batchCount == batchSize):
#             batchCount = 0
#             optimizer.step()
#             optimizer.zero_grad()
#             print("Updated\n")
#             loss_per_batch.append(sum(loss_in_batch)/len(loss_in_batch))
#             loss_in_batch.clear()
#         optimizer.step()
#   lossGraph(loss_per_batch)


In [ ]:
# def trainingClassifier(model,X,Y,learningRate,epochs, breakOutLimit, breakOut):
#     criterion = nn.CrossEntropyLoss()
#     # optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
#     minLoss = 100
#     count = 0
#     for _ in range(epochs):
#       for i in range(len(X)):
#         if(breakOut and count >= breakOutLimit):
#           break
#         optimizer.zero_grad()
#         Y_hat = model(X.iloc[i][0],X.iloc[i][1],X.iloc[i][2],X.iloc[i][3])
#         #X[0] = StudentAns, X[1] = Question, X[2] = Ref Ans, X[3] = Fullmarks
#         print("Yhat:",Y_hat," Y:",Y.iloc[i])
#         loss = criterion(Y_hat,torch.tensor([Y.iloc[i]]))
#         if(loss < minLoss):
#           count = 0
#           minLoss = loss
#         else:
#           count += 1
#         print("Loss:",loss,"Type:",type(loss))
#         loss.backward()
#         optimizer.step()


In [ ]:
import matplotlib.pyplot as plt
def lossGraph(loss_per_batch):
  plt.plot(np.linspace(1,len(loss_per_batch),len(loss_per_batch)).astype(int),loss_per_batch)

In [ ]:
def trainingClassifierBatch(model,X,Y,learningRate,epochs, breakOutLimit, breakOut,batchSize):
    criterion = nn.CrossEntropyLoss()
    # optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)
    optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
    loss_per_batch = []
    loss_in_batch = []
    for _ in range(epochs):
      batchCount = 0
      loss_in_batch.clear()
      optimizer.zero_grad()
      for i in range(len(X)):
         # model.train()
        Y_hat = model(X.iloc[i]["EssayText"],X.iloc[i]["questions"],X.iloc[i]["right_ans"],X.iloc[i]["full_marks"])
        #X[0] = StudentAns, X[1] = Question, X[2] = Ref Ans, X[3] = Fullmarks
        print("Yhat:",Y_hat," Y:",Y.iloc[i])
        loss = criterion(Y_hat,torch.tensor([Y.iloc[i]]))
        print("Loss:",loss,"Type:",type(loss))
        loss_in_batch.append(loss)
        loss.backward()
        batchCount += 1
        if(batchCount == batchSize):
          batchCount = 0
          optimizer.step()
          optimizer.zero_grad()
          print("Updated\n")
          loss_per_batch.append(sum(loss_in_batch)/len(loss_in_batch))
          loss_in_batch.clear()
      optimizer.step()
    lossGraph(loss_per_batch)
    

In [ ]:
def TrainValidateTest(df):
    train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])
    return train, validate, test

In [ ]:
def calculate(df,i):
  from collections import defaultdict
  from collections import defaultdict
  
# Initialize dictionary
  tn = defaultdict(int)
  fp = defaultdict(int)
  fn = defaultdict(int)
  tp = defaultdict(int)
  acc = 0
  precision={}
  recall={}
  for key in range(i):
    for index,row in df.iterrows():
      if(row["Real"]== key and row["Predicted"]==key):
        acc += 1
        if key not in tp:
          tp[key]=1
        else:
          tp[key]=tp[key]+1
      if(row["Real"]!=key and row["Predicted"]!=key):
        if key not in tn:
          tn[key]=1
        else:
          tn[key]=tn[key]+1
      if(row["Real"]==key and row["Predicted"]!=key):
        if key not in fn:
          fn[key]=1
        else:
          fn[key]=fn[key]+1
      if(row["Real"]!=key and row["Predicted"]==key):
        if key not in fp:
          fp[key]=1
        else:
          fp[key]=fp[key]+1
  precision = {}
  recall={}
  f1 = defaultdict(int)
  for key in range(i):
    try:
      precision[key]=tp[key]/(tp[key]+fp[key])
    except:
      precision[key]=0
    print("Precision is for",key ,"marks",precision[key])
    try:
      recall[key]=tp[key]/(tp[key]+fn[key])
    except:
      recall[key]=0 
    print("recall for", key,"is,",recall[key])
    try:
      f1[key]=(2*precision[key]*recall[key])/(precision[key]+recall[key])
    except:
      f1[key]=0
    print("f1 for",key,"is,",f1[key])  
  print("Accuracy : ",acc/len(df))

In [ ]:
def PredictTest(model,df_test):
  actual = []
  predicted = []
 
  with torch.no_grad():
    for index,row in df_test.iterrows():
        # model.eval()
        Y_Hat = model(row["EssayText"],row["questions"],row["right_ans"],row["full_marks"])
        #X[0] = StudentAns, X[1] = Question, X[2] = Ref Ans, X[3] = Fullmarks
        pred = torch.argmax(Y_Hat)
        print("Pred:",pred," Actual:",row["Score1"],"\n")
        predicted.append(pred)
        actual.append(row["Score1"])

  intermediate = {"Real":actual,"Predicted":predicted}
  df_check = pd.DataFrame(intermediate)
  calculate(df_check,4)

In [ ]:
#Creating the Dataset

# df_new = df[(df["full_marks"] == 3) & (df["EssaySet"] != 7)]
df_new = df[(df["full_marks"] == 3) & (df["EssaySet"] != 2)]
# df_new["Score1"] = df_new["Score1"]/df_new["full_marks"]
train,validate,test = TrainValidateTest(df_new)
df_y = train["Score1"]
df_x = train[["EssayText","questions","right_ans","full_marks"]]

In [ ]:
# test.iloc[3]["EssayText"] = "The president of the US is Joe Biden"
# test.iloc[3]["full_marks"] = 3
# test.iloc[3]["right_ans"] = "Joe Biden is the president of the US"
# test.iloc[3]["questions"] = "Who is the president of the US?"
# test.iloc[3]["Score1"] = 3

In [ ]:
# Get embedding sizes for every question #7
# q = []
# for i in range(1,11):
#   q.append(df[df['EssaySet'] == i]['questions'].iloc[0])
#   print(df[df['EssaySet'] == i]['questions'].iloc[0])
# b_emb = BertEmbedding()
# for i in q:
#   print(b_emb.getEmbeddings(i).size())

trainingClassifier(model,df_x,df_y,10)

In [ ]:
# trainingClassifier(model,df_x,df_y,learningRate = 0.0001,epochs = 3, breakOutLimit = 9,breakOut = False)

In [ ]:
# training(model,df_x,df_y,learningRate = 0.0001,epochs = 5, breakOutLimit = 9,breakOut = False,batchSize = 1)

In [ ]:
#Classification model
trainingClassifierBatch(model,df_x,df_y,learningRate = 0.001,epochs = 5, breakOutLimit = 9,breakOut = False,batchSize = 20)
#Can change : Learning Rate, Batch Size, Epochs

In [ ]:
# model_save_name = 'classifierTrial2.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}" 
# torch.save(model.state_dict(), path)

In [ ]:
# model_save_name = 'classifierTrial2.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}"
# model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
model_save_name = 'classifierTrial2.pt'
path = "./{model_save_name}"
model.load_state_dict(torch.load(path))

In [ ]:
  PredictTest(model,test)

Pred: tensor(3)  Actual: 3 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 1 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(3)  Actual: 2 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(3)  Actual: 3 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(3)  Actual: 1 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 1 

Pred: tensor(0)  Actual: 0 

Pred: tensor(3)  Actual: 2 

Pred: tensor(0)  Actual: 0 

Pred: tensor(0)  Actual: 0 

Pred: tensor(3)  Actual: 1 

Pred: tensor(3)  Actual: 2 

Pred: tensor(0